In [45]:
import pandas as pd
import requests 
import numpy as np

In [46]:
host = 'https://api.census.gov/data'
year = '/2015'
dataset_acronym = '/acs/acs1'
g = '?get='
variables = 'NAME,B06009_005E'
location = '&for=state:*&key='
usr_key = "0d20bc54b2f22dccdd48690f3ef9c523a9b7da70"
query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
response = requests.get(query_url)
contents = response.text


In [47]:
def create_college():
    years = ["/2010","/2011", "/2012", "/2013", "/2014", "/2015", "/2016", "/2017", "/2018", "/2019", "/2021", 
             "/2022"]
    college = pd.DataFrame()
    for year in years:
        query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
        response = requests.get(query_url)
        contents = response.text
        temp = pd.read_json(contents)
        year_col = np.full(len(temp), year[1:])
        year_col = pd.DataFrame(year_col)
        temp = pd.concat([temp, year_col], axis = 1)
        temp.columns = ["State","Residents with Bachelor's Degree or Higher","state", "Year"]
        temp = temp.sort_values("State")
        temp = temp[temp["State"] != "District of Columbia"]
        temp = temp[temp["State"] != "Puerto Rico"]
        college = pd.concat([college, temp])
    return college

In [48]:
college = create_college()
college

,State,Residents with Bachelor's Degree or Higher,state,Year
1,Alabama,439863,01,2010
2,Alaska,83346,02,2010
3,Arizona,691889,04,2010
4,Arkansas,252888,05,2010
5,California,4610875,06,2010
...,...,...,...,...
47,Virginia,1409942,51,2022
48,Washington,1302707,53,2022
49,West Virginia,188793,54,2022
50,Wisconsin,885548,55,2022


In [49]:
college = college.drop("state", axis =1)

In [50]:
college = college[college["State"] != 'NAME']

In [51]:
college

,State,Residents with Bachelor's Degree or Higher,Year
1,Alabama,439863,2010
2,Alaska,83346,2010
3,Arizona,691889,2010
4,Arkansas,252888,2010
5,California,4610875,2010
...,...,...,...
47,Virginia,1409942,2022
48,Washington,1302707,2022
49,West Virginia,188793,2022
50,Wisconsin,885548,2022


In [52]:
count = pd.to_numeric(college["Residents with Bachelor's Degree or Higher"])

In [53]:
college["Residents with Bachelor's Degree or Higher"] = count

In [54]:
college

,State,Residents with Bachelor's Degree or Higher,Year
1,Alabama,439863,2010
2,Alaska,83346,2010
3,Arizona,691889,2010
4,Arkansas,252888,2010
5,California,4610875,2010
...,...,...,...
47,Virginia,1409942,2022
48,Washington,1302707,2022
49,West Virginia,188793,2022
50,Wisconsin,885548,2022


In [55]:
college.to_csv("state_college_degree.csv", index = False)